In [1]:
import glob
import pandas as pd
from pylab import *
from palettable.colorbrewer.qualitative import Set2_7
import h5py
from pathlib import Path


In [2]:
colors = Set2_7.mpl_colors

params = {
    'axes.labelsize': 8,
    'font.size': 8,
    'legend.fontsize': 10,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'text.usetex': False,
    'figure.figsize': [4.5, 4.5]
}
rcParams.update(params)

# Fig 1: 变转速时频、频谱绘制

In [21]:
data_path = Path("D:\datasets\复合故障数据集\数据\Case 1")
file_path = data_path / "B4L.h5"
read_file = h5py.File(file_path, 'r')
signals = pd.DataFrame(np.array(read_file['x']))
# read_file.close()
labels = pd.DataFrame(np.array(read_file['label']))
labels

,0,1,2,3,4,5
0,1,0,0,0,0,0
1,1,0,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,0,1,0,0,0
5,0,0,1,0,0,0
6,0,0,0,1,0,0
7,0,0,0,1,0,0
8,0,0,0,0,1,0
9,0,0,0,0,1,0
